In [1]:
import os
import numpy as np
import glob
from tess_sip import SIP
from astropy.coordinates import SkyCoord
from astropy import units as u
import lightkurve as lk
from astroquery.mast import Tesscut
import tess_cpm
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

sys.path.insert(0, "../")
sys.path.insert(0, "../../Packages/eleanor/")
import eleanor

import warnings
warnings.filterwarnings('ignore')

from michael import janet

In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

index = uni[uni.source_id == int(5310986619291928704)].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
j = janet.boot(uni, index=index, pipeline='tess-sip-detrended')
j.run()
plt.show()

In [ ]:
j = janet.boot(uni, index=index, pipeline='eleanor')
j.run()
plt.show()

In [ ]:
j = janet.boot(uni, index=index, pipeline='unpopular')
j.run()
plt.show()

In [ ]:
j.sectorlist[0]

In [ ]:
list(j.void)

In [ ]:
fig, ax= plt.subplots()
sector0 = j.sectorlist[0]
s0 = j.sectors[0]
ax.set_title(f'Frame 0 Sector {sector0}')
ax.imshow(np.log10(j.void[f'tpfs_{s0}'][0].flux.value[0]), zorder=1)

In [ ]:
sys.exit()

In [ ]:
coords = SkyCoord(ra = j.ra, dec = j.dec, unit = (u.deg, u.deg))


In [ ]:
from michael.data import data_class
d = data_class(j)
d.setup_data()

In [ ]:
d.build_unpopular_lc()


In [ ]:
sfiles = []
for sector in j.sectors:
    split = sector.split('-')
    if len(split) > 1:
        sfiles = []
        for s in np.arange(int(split[0]), int(split[1])+1):
            strlen = np.floor(np.log10(s)).astype(int)+1
            secstr = 's0000'[:-strlen] + str(s)

            sfile = glob.glob(f'{d.path}'+
                                f'*{secstr}*astrocut.fits')

            if len(sfile) == 0:
                raise ValueError("No tesscut files could be found for this target.")

            sfiles.append(sfile[0])

        tpflist = [lk.TessTargetPixelFile(f).cutout([26,26],13) for f in sfiles]
        tpfs = lk.TargetPixelFileCollection(tpflist)

In [ ]:
img = tpfs[1].plot();
plt.show()

In [ ]:
lims = img.get_images()[0].get_clim()

In [ ]:
plt.imshow(j.void['datum_57'].tpf[0], origin='lower',
          vmin = lims[0], vmax = lims[1])
pix = np.where(j.void[f'datum_57'].aperture > 0)
plt.scatter(pix[0], pix[1], edgecolors='w', lw=5, marker=',', facecolors='none', s=600, zorder=2, label='Aperture')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for i, m in enumerate(['c','r','cpm','rdt', 'pca','corn']):
    (j.void[f'{m}lc_16-17'] + i*.02).plot(ax=ax, label=m)
plt.show()

In [ ]:
fig, ax = plt.subplots()
for i, m in enumerate(['c','r','cpm','rdt', 'pca','corn']):
    (j.void[f'{m}lc_56-57'] + i*.02).plot(ax=ax, label=m)
plt.show()

In [ ]:
pipelines = {'eleanor' : 'c',
             'eleanor-pca' : 'pca',
             'eleanor-corner' : 'corn',
             'unpopular' : 'cpm',
             'tess-sip': 'r',
             'tess-sip-detrended' : 'rdt'}

In [ ]:
for p in list(pipelines):
    j.pipeline = p
    j.pl = pipelines[j.pipeline]
    j.run()
    plt.show()

In [ ]:
len(uni)

In [ ]:
j.pipeline